In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
%matplotlib inline

In [ ]:
nodes = np.linspace(0, 1, 5)
print nodes

In [ ]:
def f(t): return np.sin(3*t)

In [ ]:
x = np.linspace(0, 1, 100)                    # the domain

In [ ]:
interpolant = interp1d(nodes, f(nodes), kind='nearest')

In [ ]:
plt.rc('text', usetex=True)
plt.figure(figsize=(15,3.75))

plt.axes().set_aspect('equal')
plt.plot(nodes, f(nodes), 'ro', label='Nodes')
plt.plot(x, f(x), 'r-', label=r'f(x) = \sin(3x)')
plt.plot(x, interpolant(x), 'b--', label='Interpolation')
plt.title("Nearest-neighbor approximation")
plt.legend(loc=2)
plt.ylim(-0.05, 1.05)
plt.xlim(-0.5, 1.05)

plt.show()

In [ ]:
from scipy.interpolate import BarycentricInterpolator, barycentric_interpolate, lagrange

In [ ]:
def f(t): return 1. / (1 + t**2)
x = np.linspace(-5,5,1000)
nodes = np.linspace(-5, 5, 11)
interpolant = BarycentricInterpolator(nodes, f(nodes))

plt.figure(figsize=(12,6))

plt.subplot(121, aspect='auto')
plt.plot(x, interpolant(x), 'b--', label="Lagrange Interpolation")
plt.plot(nodes, f(nodes), 'ro', label='nodes')
plt.plot(x, f(x), 'r-', label="original")
plt.legend(loc=9)
plt.title("11 equally distributed nodes")

newnodes = np.linspace(-4.5, 4.5, 10)
print 'nodes = ', nodes
print 'newnodes = ', newnodes

interpolant.add_xi(newnodes, f(newnodes))

plt.subplot(122, aspect='auto')
plt.plot(x, interpolant(x), 'b--', label="Lagrange Interpolation")
plt.plot(nodes, f(nodes), 'ro', label='nodes')
plt.plot(x, f(x), 'r-', label="original")
plt.legend(loc=8)
plt.title("21 equally spaced nodes")
plt.show()

In [ ]:
print interpolant.xi

In [ ]:
print interpolant.yi

In [ ]:
y = barycentric_interpolate(nodes, f(nodes), x)

In [ ]:
g = lagrange(nodes, f(nodes))
print g

In [ ]:
from scipy.interpolate import KroghInterpolator, krogh_interpolate

In [ ]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
nodes = np.linspace(-1, 1, 10)
x = np.linspace(-1, 1, 1000)
xi = np.repeat(nodes, 2)
print xi
yi = np.ravel(np.dstack((np.abs(nodes), np.sign(nodes))))
print yi

In [ ]:
interpolant = KroghInterpolator(xi, yi)

In [ ]:
plt.figure()
plt.axes().set_aspect('equal')
plt.plot(x, interpolant(x), 'b', label='Hermite Interpolation')
plt.plot(nodes, np.abs(nodes), 'ro')
plt.plot(x, np.abs(x), 'r', label='original')
plt.legend(loc=9)
plt.title('Bernstein example')
plt.show()

In [ ]:
from scipy.interpolate import PiecewisePolynomial

nodes = np.linspace(0, 1, 5)
x = np.linspace(0, 1, 100) 

def f(t): return np.sin(3 * t)

interpolant = interp1d(nodes, f(nodes), kind='linear')

plt.figure(figsize=(12,6))
plt.subplot(121, aspect='equal')
plt.plot(x, interpolant(x), 'b--', label="interpolation")
plt.plot(nodes, f(nodes), 'ro')
plt.plot(x, f(x), 'r-', label="original")
plt.legend(loc=8)
plt.title("Piecewise Linear Interpolation")

yi = np.zeros((len(nodes), 2))
yi[:,0] = f(nodes)
yi[:,1] = 3 * np.cos(3 * nodes)
print yi

interpolant = PiecewisePolynomial(nodes, yi, orders=2)

plt.subplot(122, aspect='equal')
plt.plot(x, interpolant(x), 'b--', label="interpolation")
plt.plot(nodes, f(nodes), 'ro')
plt.plot(x, f(x), 'r-', label="original")
plt.legend(loc=8)
plt.title("Piecewise Quadratic interpolation")
plt.show()

In [ ]:
print np.abs(f(x) - interpolant(x)).max()

f1prime = lambda t: 3 * np.cos(3 * t)
print np.abs(f1prime(x) - interpolant.derivatives(x)).max()

f2prime = lambda t: -9 * np.sin(3 * x)
print np.abs(f2prime(x) - interpolant.derivatives(x,der=2)).max()

In [ ]:
interpolant = PiecewisePolynomial(nodes, yi, orders=[1,3,3,1])

In [ ]:
interpolant.append(1.25, np.array([f(1.25)]))

In [ ]:
splines = ['zero', 'slinear', 'quadratic', 'cubic', 4, 5]
g = KroghInterpolator([0,0,0,1,1,1,2,2,2,3,3,3], [10,0,0,1,0,0,0.25,0,0,0.125,0,0])
def f(t): return np.log1p(g(t))
x = np.linspace(0,3,100)
nodes = np.linspace(0,3,11)

plt.figure(figsize=(16,8))

for k in xrange(6):
    interpolant = interp1d(nodes, f(nodes), kind = splines[k])
    plt.subplot(2,3,k+1, aspect='equal')
    plt.plot(nodes, f(nodes), 'ro')
    plt.plot(x, f(x), 'r-', label='original')
    plt.plot(x, interpolant(x), 'b--', label='interpolation')
    plt.title('{0} spline'.format(splines[k]))
    plt.legend()

plt.show()

In [ ]:
from scipy.interpolate import PchipInterpolator

interpolant = PchipInterpolator(nodes, f(nodes))

plt.figure()
plt.axes().set_aspect('equal')
plt.plot(nodes, f(nodes), 'ro')
plt.plot(x, f(x), 'r-', label='original')
plt.plot(x, interpolant(x), 'b--', label='interpolation')
plt.title('PCHIP interpolation')
plt.legend()
plt.show()

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

interpolant = InterpolatedUnivariateSpline(nodes, f(nodes), k=5)

area = interpolant.integral(0,3)
print area